In [ ]:
import json
from glob import glob
from pprint import pprint
from tqdm import tqdm
import inflect

p = inflect.engine()

In [ ]:
with open("./ambiguous_questions.json") as f:
    ambiguous_questions = json.load(f)

with open("./ambiguous_questions_rebuilt.json") as f:
    ambiguous_questions_pr = json.load(f)

In [ ]:
with open("./sceneGraphs/train_sceneGraphs.json") as f:
    sceneGraphs = json.load(f)

In [ ]:
def get_same_name_entities(sceneGraph, target_object_id):
    
    entities = []
    
    target_object_name = sceneGraph['objects'][target_object_id]['name']
    for object_key, object_value in sceneGraph["objects"].items():
        if object_key != target_object_id and object_value['name'] == target_object_name:
            entities.append((object_key, object_value))
            
    return entities

In [ ]:
def make_unique_attribute_question(sceneGraph, target_object_id):
    
    questions = []
    
    target_object = sceneGraph["objects"][target_object_id]
    
    same_entities = get_same_name_entities(sceneGraph, target_object_id)
    
    target_attributes = set(target_object["attributes"])
    
    question_type = "attribute"
    
    if len(same_entities) == 0:
        return questions
    
    # print(target_attributes)
    
    for entity_key, entity_value in same_entities:
        target_attributes = target_attributes - set(entity_value['attributes'])
        
    yes_unique_attributes = target_attributes
    
    # print(target_attributes)
    
    be_verb = "Is" if p.singular_noun(target_object['name']) == False else "Are"
    
    for unique_attribute in yes_unique_attributes:
        
        intermediate_question = f"{be_verb} the {target_object['name']} {unique_attribute}?"
        intermediate_answer = "yes"

        questions.append({'question' : intermediate_question,
                          'answer' : intermediate_answer,
                          'type': question_type,
                          'label' : "positive"})
    
    intersection = None
    for entity_key, entity_value in same_entities:
        
        if intersection is None:
            intersection = set(entity_value['attributes'])
        else:
            intersection = intersection & set(entity_value['attributes'])
            
    if intersection is not None:
        no_unique_attributes = intersection - set(target_object["attributes"])

        # print(no_unique_attributes)
        
        for unique_attribute in no_unique_attributes:
            
            intermediate_question = f"{be_verb} the {target_object['name']} {unique_attribute}?"
            intermediate_answer = "no"

            questions.append({'question' : intermediate_question,
                            'answer' : intermediate_answer,
                            'type': question_type,
                            'label' : "positive"})
        
    return questions

In [ ]:
def relations2set(relations):
    
    values = []
    for relation in relations:
        name = relation['name']
        object = relation['object']
        
        values.append((name, object))
        
    return set(values)

def make_unique_position_question(sceneGraph, unique_relations, target_object_name):
    
    be_verb = "Is" if  p.singular_noun(target_object_name) == False else "Are"
    
    left_count = 0
    right_count = 0
    remove_relations = []
    for unique_relation in unique_relations:
        relation = unique_relation[0]
        object_name = sceneGraph["objects"][unique_relation[1]]['name']
        
        if object_name == target_object_name:
            if "left" in relation:
                left_count += 1
                remove_relations.append(unique_relation)
            elif "right" in relation:
                right_count += 1
                remove_relations.append(unique_relation)
            else:
                pass
    
    for remove_relation in remove_relations:
        unique_relations.remove(remove_relation)
    
    if left_count + right_count > 1:
        object_name = p.plural(target_object_name)
    else:
        object_name = target_object_name
    
    position_relation = ""
    intermediate_answer = "yes"
    question_type = 'relation'
    
    
    if left_count != 0 and right_count == 0:
        position_relation = "on the left of"
        intermediate_question = f"{be_verb} the {target_object_name} {position_relation} the other {object_name}?"
        
    elif left_count == 0 and right_count != 0:
        position_relation = "on the right of"
        intermediate_question = f"{be_verb} the {target_object_name} {position_relation} the other {object_name}?"
    elif left_count != 0 and right_count != 0 and left_count == right_count:
        position_relation  = "in the middle of"
        intermediate_question = f"{be_verb} the {target_object_name} {position_relation} the other {object_name}?"
    else:
        pass
    
    if position_relation == "":
        return None, unique_relations 
    else:
        question = {"question" : intermediate_question,
                    "answer" : intermediate_answer,
                    'type' : question_type,
                    'label' : "positive"}
        return question, unique_relations

def make_unique_relation_question(sceneGraph, target_object_id):
    
    questions = []
    
    target_object = sceneGraph["objects"][target_object_id]
    # pprint(target_object)
    
    same_entities =  get_same_name_entities(sceneGraph, target_object_id)
    
    # pprint(same_entities)
    
    if len(same_entities) == 0:
        return questions
    
    target_relations = relations2set(target_object["relations"])
    
    question_type = "relation"
    
    if len(target_relations) > 0 :
        postion_question, yes_unique_relations = make_unique_position_question(sceneGraph, target_relations, target_object['name'])
    
        if postion_question is not None:
            questions.append(postion_question)

        target_relations = yes_unique_relations
    
    for entity_key, entity_value in same_entities:
        target_relations = target_relations - relations2set(entity_value['relations'])
        
    yes_unique_relations = target_relations
    
    be_verb = "Is" if p.singular_noun(target_object['name']) == False else "Are"
    
    for unique_relation in yes_unique_relations:
            
        relation = unique_relation[0]
        object_name = sceneGraph["objects"][unique_relation[1]]['name']
        
        if p.singular_noun(object_name) == False:
            object_name = p.a(object_name)
        
        if len(get_same_name_entities(sceneGraph, unique_relation[1])) == 0:
        
            intermediate_question = f"{be_verb} the {target_object['name']} {relation} {object_name}?"
            intermediate_answer = "yes"

            questions.append({'question' : intermediate_question,
                            'answer' : intermediate_answer,
                            'type': question_type,
                            'label' : "positive"})
    
    intersection = None
    for entity_key, entity_value in same_entities:
        
        if intersection is None:
            intersection =  relations2set(entity_value['relations'])
        else:
            intersection = intersection & relations2set(entity_value['relations'])
            
    if intersection is None:
        no_unique_relations = set()
    else:
        no_unique_relations = intersection - relations2set(target_object["relations"])
        
        # print(no_unique_relations)
        
    for unique_relation in no_unique_relations:
        
        relation = unique_relation[0]
        object_name = sceneGraph["objects"][unique_relation[1]]['name']
        
        if p.singular_noun(object_name) == False:
            object_name = p.a(object_name)
        
        if len(get_same_name_entities(sceneGraph, unique_relation[1])) == 0:
        
            intermediate_question = f"{be_verb} the {target_object['name']} {relation} {object_name}?"
            intermediate_answer = "no"
            
            questions.append({'question' : intermediate_question,
                            'answer' : intermediate_answer,
                            'type': question_type,
                            'label' : "positive"
                            })
        
    return questions

In [ ]:
def make_negative_attribute_question(sceneGraph, target_object_id):
    
    questions = []
    
    target_object = sceneGraph["objects"][target_object_id]
    
    same_entities = get_same_name_entities(sceneGraph, target_object_id)
    
    target_attributes = set(target_object["attributes"])
    
    question_type = "attribute"
    
    if len(same_entities) == 0:
        return questions
    
    # print(target_attributes)
    
    union_set = set()
    for entity_key, entity_value in same_entities:
        
        union_set = union_set.union(set(entity_value['attributes']))
       
    yes_negative_attributes = union_set.intersection(target_attributes)
        
    
    # print(target_attributes)
    
    be_verb = "Is" if p.singular_noun(target_object['name']) == False else "Are"
    
    for unique_attribute in yes_negative_attributes:
        
        intermediate_question = f"{be_verb} the {target_object['name']} {unique_attribute}?"
        intermediate_answer = "yes"

        questions.append({'question' : intermediate_question,
                          'answer' : intermediate_answer,
                          'type': question_type,
                          'label' : "negative"})
    
    each_unique_attributes = set()
    if len(same_entities) > 1:
        for entity_key, entity_value in same_entities:
            
            entity_attributes = set(entity_value['attributes'])
            for other_entity_key, other_entity_value in same_entities:
                if entity_key != other_entity_key:
                    entity_attributes = entity_attributes - set(other_entity_value['attributes'])
            
            unique_entity_attributes = entity_attributes
            
            each_unique_attributes.update(unique_entity_attributes)
            
                
        if len(each_unique_attributes) != 0:
            no_negative_attributes = each_unique_attributes - set(target_object["attributes"])

            # print(no_unique_attributes)
            
            for unique_attribute in no_negative_attributes:
                
                intermediate_question = f"{be_verb} the {target_object['name']} {unique_attribute}?"
                intermediate_answer = "no"

                questions.append({'question' : intermediate_question,
                                'answer' : intermediate_answer,
                                'type': question_type,
                                'label' : "negative"})
        
    return questions

In [ ]:
def make_negative_position_question(sceneGraph, unique_relations, target_object_name):
    
    questions = []
    
    be_verb = "Is" if p.singular_noun(target_object_name) == False else "Are"
    
    left_count = 0
    right_count = 0
    remove_relations = []
    for unique_relation in unique_relations:
        relation = unique_relation[0]
        object_name = sceneGraph["objects"][unique_relation[1]]['name']
        
        if object_name == target_object_name:
            if "left" in relation:
                left_count += 1
                remove_relations.append(unique_relation)
            elif "right" in relation:
                right_count += 1
                remove_relations.append(unique_relation)
            else:
                pass
    
    for remove_relation in remove_relations:
        unique_relations.remove(remove_relation)
    
    if left_count + right_count > 1:
        object_name = p.plural(target_object_name)
    else:
        object_name = target_object_name
    
    position_relation = ""
    intermediate_answer = "yes"
    question_type = 'relation'
    
    
    if left_count != 0 and right_count == 0:
        pass
    elif left_count == 0 and right_count != 0:
        pass
    elif left_count != 0 and right_count != 0 and left_count == right_count:
        pass    
    else:
        if left_count != 0:
            position_relation  = "on the left of"
            intermediate_question = f"{be_verb} the {target_object_name} {position_relation} the other {object_name}?"
            questions.append({"question" : intermediate_question,
                    "answer" : intermediate_answer,
                    'type' : question_type,
                    'label' : "negative"})
        if right_count != 0:
            position_relation  = "on the right of"
            intermediate_question = f"{be_verb} the {target_object_name} {position_relation} the other {object_name}?"
            questions.append({"question" : intermediate_question,
                    "answer" : intermediate_answer,
                    'type' : question_type,
                    'label' : "negative"})
        if left_count != 0 and right_count != 0 and left_count != right_count:
            position_relation  = "in the middle of"
            intermediate_question = f"{be_verb} the {target_object_name} {position_relation} the other {object_name}?"
            questions.append({"question" : intermediate_question,
                    "answer" : intermediate_answer,
                    'type' : question_type,
                    'label' : "negative"})
    
    if len(questions) == 0:
        return None, unique_relations 
    else:
        
        return questions, unique_relations

def make_negative_relation_question(sceneGraph, target_object_id):
    
    questions = []
    
    target_object = sceneGraph["objects"][target_object_id]
    # pprint(target_object)
    
    same_entities =  get_same_name_entities(sceneGraph, target_object_id)
    
    # pprint(same_entities)
    
    if len(same_entities) == 0:
        return questions
    
    target_relations = relations2set(target_object["relations"])
    
    question_type = "relation"
    
    if len(target_relations) > 0 :
        postion_question, yes_negative_relations = make_negative_position_question(sceneGraph, target_relations, target_object['name'])
    
        if postion_question is not None:
            questions.extend(postion_question)

        target_relations = yes_negative_relations
    
    union_relations = None
    for entity_key, entity_value in same_entities:
        if union_relations is None:
            union_relations = relations2set(entity_value['relations'])
        else:
            union_relations = union_relations.union(relations2set(entity_value['relations']))
    
    for entity_key, entity_value in same_entities:
        target_relations = target_relations.intersection(union_relations)
        
    yes_negative_relations = target_relations
    
    be_verb = "Is" if p.singular_noun(target_object['name']) == False else "Are"
    
    for unique_relation in yes_negative_relations:
            
        relation = unique_relation[0]
        object_name = sceneGraph["objects"][unique_relation[1]]['name']
        
        if p.singular_noun(object_name) == False:
            object_name = p.a(object_name)
        
        if len(get_same_name_entities(sceneGraph, unique_relation[1])) == 0:
        
            intermediate_question = f"{be_verb} the {target_object['name']} {relation} {object_name}?"
            intermediate_answer = "yes"

            questions.append({'question' : intermediate_question,
                            'answer' : intermediate_answer,
                            'type': question_type,
                            'label' : "negative"})
    
    intersection = None
    each_entity_unique_relations = set()
    if len(same_entities) > 1:
        for entity_key, entity_value in same_entities:
            
            entity_relations = relations2set(entity_value['relations']).copy()
            for other_entity_key, other_entity_value in same_entities:
                if entity_key != other_entity_key:
                    entity_relations = entity_relations - relations2set(other_entity_value['relations'])
                    
            each_entity_unique_relations.update(entity_relations)
            
        if len(each_entity_unique_relations) == 0:
            no_negative_relations = set()
        else:
            no_negative_relations = each_entity_unique_relations - relations2set(target_object["relations"])
            
            # print(no_unique_relations)
            
        for negative_relation in no_negative_relations:
            
            relation = negative_relation[0]
            object_name = sceneGraph["objects"][negative_relation[1]]['name']
            
            if p.singular_noun(object_name) == False:
                object_name = p.a(object_name)
            
            if len(get_same_name_entities(sceneGraph, negative_relation[1])) == 0:
            
                intermediate_question = f"{be_verb} the {target_object['name']} {relation} {object_name}?"
                intermediate_answer = "no"
                
                questions.append({'question' : intermediate_question,
                                'answer' : intermediate_answer,
                                'type': question_type,
                                'label' : "negative"
                                })
        
    return questions
    

In [ ]:
intermedidate_questions = {}
question_list = []
negative_sample_list = []
for idx, (qid, example) in tqdm(enumerate(ambiguous_questions.items()), total=len(ambiguous_questions.items())):
   
    image_id = example['imageId']
    ambiguous_question = ambiguous_questions_pr[qid]
    if 'irrelated_object_names' in example.keys():
        example['ambiguous_entities'] = example.pop('irrelated_object_names')
    ambiguous_entity_keys = list(example['ambiguous_entities'].keys())
    
    # print(example)    
    scene_graph = sceneGraphs[image_id]
    
    target_objects = list(example['annotations']['question'].values())
    
    # print(image_id)
    # print(ambiguous_question)
    unique_ambigous_entities = set(example['ambiguous_entities'].values())
    
    #ambiguous_object_questions = {}
    if image_id not in intermedidate_questions.keys():
        intermedidate_questions[image_id] = {}
    
    for target_object_key in target_objects:
        target_object_name = scene_graph['objects'][target_object_key]['name']
        intermediate_question_list = []
       
        if target_object_name in ambiguous_question['question'] and target_object_key not in intermedidate_questions[image_id].keys():
    
            attribute_questions = make_unique_attribute_question(scene_graph, target_object_key)
            relation_questions = make_unique_relation_question(scene_graph, target_object_key)
            negative_attribute_questions = make_negative_attribute_question(scene_graph, target_object_key)
            negative_relation_questions = make_negative_relation_question(scene_graph, target_object_key)
            
            if len(attribute_questions) > 0:
                intermediate_question_list += attribute_questions
            if len(relation_questions) > 0:
                intermediate_question_list += relation_questions
            if len( negative_attribute_questions) > 0:
                intermediate_question_list += negative_attribute_questions
                negative_sample_list  += negative_attribute_questions
            if len(negative_relation_questions) > 0:
                intermediate_question_list += negative_relation_questions
                negative_sample_list  += negative_relation_questions
            
            if len(intermediate_question_list) > 0:
                question_list += intermediate_question_list
                intermedidate_questions[image_id][target_object_key] = intermediate_question_list
            
            # else:
            #     intermedidate_questions[image_id][target_object_key] = []
            #     # print(target_object_key)

print(len(question_list))
print(len(negative_sample_list))
with open("intermediate_questions.json", 'w') as f:
    json.dump(intermedidate_questions,f)

In [ ]:
print(len(question_list))
count =len([len(intermedidate_questions[intermedidate_question_key].keys()) for intermedidate_question_key in intermedidate_questions.keys()])
print(len([len(intermedidate_questions[intermedidate_question_key].keys()) for intermedidate_question_key in intermedidate_questions.keys()]))
print(len(question_list) / count)

In [ ]:
relation_set = set()
for image_graph_key, image_graph_value in tqdm(sceneGraphs.items()):
    for object_key, object_value in image_graph_value['objects'].items():
        for relation in (object_value['relations']):
            relation_name = relation['name']
            relation_set.add(relation_name)
            
print(len(relation_set))
print(relation_set)